<a href="https://colab.research.google.com/github/dondreojordan/DS-Unit-2-Linear-Models/blob/master/module3-ridge-regression/LS_DS_213_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [4]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [5]:
'''Quick Observations: Nan values, no sales price.'''

df.head(20)

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
0,1,CHELSEA,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,NaN,R4,"447 WEST 18TH STREET, PH12A",PH12A,10011.0,1.0,0.0,1.0,"10,733",1979.0,2007.0,2,R4,0,01/01/2019
1,1,FASHION,21 OFFICE BUILDINGS,4,812,68,NaN,O5,144 WEST 37TH STREET,NaN,10018.0,0.0,6.0,6.0,"2,962",15435.0,1920.0,4,O5,0,01/01/2019
2,1,FASHION,21 OFFICE BUILDINGS,4,839,69,NaN,O5,40 WEST 38TH STREET,NaN,10018.0,0.0,7.0,7.0,"2,074",11332.0,1930.0,4,O5,0,01/01/2019
3,1,GREENWICH VILLAGE-WEST,13 CONDOS - ELEVATOR APARTMENTS,2,592,1041,NaN,R4,"1 SHERIDAN SQUARE, 8C",8C,10014.0,1.0,0.0,1.0,0,500.0,0.0,2,R4,0,01/01/2019
4,1,UPPER EAST SIDE (59-79),15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,1379,1402,NaN,R1,"20 EAST 65TH STREET, B",B,10065.0,1.0,0.0,1.0,0,6406.0,0.0,2,R1,0,01/01/2019
5,1,UPPER EAST SIDE (79-96),07 RENTALS - WALKUP APARTMENTS,2B,1551,131,NaN,C4,354 EAST 89TH STREET,NaN,10128.0,10.0,0.0,10.0,"2,013",6570.0,1920.0,2,C4,0,01/01/2019
6,1,UPPER WEST SIDE (96-116),07 RENTALS - WALKUP APARTMENTS,2B,1891,159,NaN,C4,304 WEST 106 STREET,NaN,10025.0,10.0,0.0,10.0,"1,716",5810.0,1900.0,2,C4,0,01/01/2019
7,2,MORRIS PARK/VAN NEST,01 ONE FAMILY DWELLINGS,1,4090,37,NaN,A1,1193 SACKET AVENUE,NaN,10461.0,1.0,0.0,1.0,"3,404",1328.0,1925.0,1,A1,0,01/01/2019
8,2,MORRIS PARK/VAN NEST,01 ONE FAMILY DWELLINGS,1,4120,18,NaN,A5,1215 VAN NEST AVENUE,NaN,10461.0,1.0,0.0,1.0,"2,042",1728.0,1935.0,1,A5,0,01/01/2019
9,2,MORRIS PARK/VAN NEST,01 ONE FAMILY DWELLINGS,1,4120,20,NaN,A5,1211 VAN NEST AVENUE,NaN,10461.0,1.0,0.0,1.0,"2,042",1728.0,1935.0,1,A5,0,01/01/2019


In [6]:
df['BOROUGH'].dtype

dtype('int64')

In [7]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [8]:
df['BOROUGH'].dtype

dtype('O')

In [9]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [10]:

top10

Index(['FLUSHING-NORTH', 'UPPER EAST SIDE (59-79)', 'UPPER EAST SIDE (79-96)',
       'BEDFORD STUYVESANT', 'BOROUGH PARK', 'UPPER WEST SIDE (59-79)',
       'GRAMERCY', 'ASTORIA', 'FOREST HILLS', 'EAST NEW YORK'],
      dtype='object')

In [14]:
pd.options.display.float_format = '{:,.0f}'.format 
# Convert values from floats to int
df.describe()
# Observe the numerical features

,BLOCK,LOT,EASE-MENT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,SALE_PRICE
count,"23,040","23,040",0,"23,039","23,039","23,039","23,039","23,039","23,005","23,040","23,040"
mean,"4,459",354,nan,"10,791",2,0,2,"3,509","1,820",2,"1,328,133"
std,"3,713",628,nan,"1,103",10,6,12,"22,186",488,1,"10,253,944"
min,1,1,nan,0,0,-148,0,0,0,1,0
25%,"1,341",22,nan,"10,306",0,0,1,494,"1,920",1,0
50%,"3,546",49,nan,"11,211",1,0,1,"1,356","1,940",1,"455,000"
75%,"6,674",375,nan,"11,360",2,0,2,"2,280","1,965",2,"875,000"
max,"16,350","9,057",nan,"11,697",750,570,755,"1,303,935","2,019",4,"850,000,000"


In [15]:
""" There are sales prices"""

' There are sales prices'

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23040 entries, 0 to 23039
Data columns (total 21 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   BOROUGH                         23040 non-null  object 
 1   NEIGHBORHOOD                    23040 non-null  object 
 2   BUILDING_CLASS_CATEGORY         23040 non-null  object 
 3   TAX_CLASS_AT_PRESENT            23039 non-null  object 
 4   BLOCK                           23040 non-null  int64  
 5   LOT                             23040 non-null  int64  
 6   EASE-MENT                       0 non-null      float64
 7   BUILDING_CLASS_AT_PRESENT       23039 non-null  object 
 8   ADDRESS                         23040 non-null  object 
 9   APARTMENT_NUMBER                5201 non-null   object 
 10  ZIP_CODE                        23039 non-null  float64
 11  RESIDENTIAL_UNITS               23039 non-null  float64
 12  COMMERCIAL_UNITS                

- [ ] Subset of the data where **BUILDING_CLASS_CATEGORY == '01 ONE FAMILY DWELLINGS'** and the sale price was **more than 100 thousand and less than 2 million.**

In [17]:
one_family = df.loc[df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS']

In [18]:
usable = one_family[(one_family.SALE_PRICE > 100000) & (one_family.SALE_PRICE < 2000000)]

In [19]:
usable.sort_values(by=['SALE_PRICE'])

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
18147,4,OTHER,01 ONE FAMILY DWELLINGS,1,11159,11,nan,A0,223-29 103RD AVENUE,NaN,"11,429",1,0,1,"3,910","1,163","1,950",1,A0,104000,03/30/2019
3489,4,OTHER,01 ONE FAMILY DWELLINGS,1,16118,56,nan,A5,313 BEACH 86 STREET,NaN,"11,693",1,0,1,"1,802","1,205","1,920",1,A5,105000,01/18/2019
19961,5,OTHER,01 ONE FAMILY DWELLINGS,1,1087,14,nan,A1,147 HARRISON AVENUE,NaN,"10,302",1,0,1,"2,525","1,336","1,899",1,A1,107500,04/09/2019
791,2,OTHER,01 ONE FAMILY DWELLINGS,1,3992,55,nan,A2,2425 ST. RAYMONDS AVENUE,NaN,"10,461",1,0,1,"3,590","1,400","1,925",1,A2,108000,01/07/2019
15768,2,OTHER,01 ONE FAMILY DWELLINGS,1D,5514,62,nan,A8,"81D EDGEWATER PARK, 81D",NaN,"10,465",0,0,0,0,0,"1,930",1,A8,110000,03/20/2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19698,4,OTHER,01 ONE FAMILY DWELLINGS,1,4509,54,nan,A1,150-57 6TH AVENUE,NaN,"11,357",1,0,1,"5,000","2,432","2,016",1,A1,1909219,04/08/2019
22112,3,OTHER,01 ONE FAMILY DWELLINGS,1,5141,88,nan,A1,296 STRATFORD ROAD,NaN,"11,218",1,0,1,"6,750","2,275","1,920",1,A1,1912500,04/23/2019
7115,3,OTHER,01 ONE FAMILY DWELLINGS,1,5050,2,nan,S0,625 ROGERS AVENUE,NaN,"11,225",1,2,3,"3,700","2,128","1,905",1,S0,1925000,02/05/2019
7554,3,OTHER,01 ONE FAMILY DWELLINGS,1,6101,11,nan,A1,1 MARINE AVENUE,NaN,"11,209",1,0,1,"4,290","1,960","1,925",1,A1,1950000,02/07/2019


- [ ] Do **train/test split**. 

>>Train: January — March 2019.

>>Test: April 2019.

In [20]:
import pandas as pd

# usable = parameters: BUILDING_CLASS_CATEGORY == '01 ONE FAMILY DWELLINGS' 
# & sale price was more than 100 thousand and less than 2 million.
usable

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,nan,A9,4832 BAY PARKWAY,NaN,"11,230",1,0,1,"6,800","1,325","1,930",1,A9,550000,01/01/2019
61,4,OTHER,01 ONE FAMILY DWELLINGS,1,7918,72,nan,A1,80-23 232ND STREET,NaN,"11,427",1,0,1,"4,000","2,001","1,940",1,A1,200000,01/01/2019
78,2,OTHER,01 ONE FAMILY DWELLINGS,1,4210,19,nan,A1,1260 RHINELANDER AVE,NaN,"10,461",1,0,1,"3,500","2,043","1,925",1,A1,810000,01/02/2019
108,3,OTHER,01 ONE FAMILY DWELLINGS,1,5212,69,nan,A1,469 E 25TH ST,NaN,"11,226",1,0,1,"4,000","2,680","1,899",1,A1,125000,01/02/2019
111,3,OTHER,01 ONE FAMILY DWELLINGS,1,7930,121,nan,A5,5521 WHITTY LANE,NaN,"11,203",1,0,1,"1,710","1,872","1,940",1,A5,620000,01/02/2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23029,4,OTHER,01 ONE FAMILY DWELLINGS,1,13215,3,nan,A2,244-15 135 AVENUE,NaN,"11,422",1,0,1,"3,300","1,478","1,925",1,A2,635000,04/30/2019
23031,4,OTHER,01 ONE FAMILY DWELLINGS,1,11612,73,nan,A1,10919 132ND STREET,NaN,"11,420",1,0,1,"2,400","1,280","1,930",1,A1,514000,04/30/2019
23032,4,OTHER,01 ONE FAMILY DWELLINGS,1,11808,50,nan,A0,135-24 122ND STREET,NaN,"11,420",1,0,1,"4,000","1,333","1,945",1,A0,635000,04/30/2019
23033,4,OTHER,01 ONE FAMILY DWELLINGS,1,12295,23,nan,A1,134-34 157TH STREET,NaN,"11,434",1,0,1,"2,500","1,020","1,935",1,A1,545000,04/30/2019


In [22]:
# y will be what we want to predict.
y = usable['SALE_PRICE']
y.shape

(3151,)

In [23]:
# X features used for the prediction.
# .drop() to remove predictor 'SALE_PRICE'
X = usable.drop('SALE_PRICE', axis=1)
X.shape

(3151, 20)

In [25]:
X.info()
# Take a look at 'SALE_DATE'. 
# Convert format from object to datetime. 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3151 entries, 44 to 23035
Data columns (total 20 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   BOROUGH                         3151 non-null   object 
 1   NEIGHBORHOOD                    3151 non-null   object 
 2   BUILDING_CLASS_CATEGORY         3151 non-null   object 
 3   TAX_CLASS_AT_PRESENT            3151 non-null   object 
 4   BLOCK                           3151 non-null   int64  
 5   LOT                             3151 non-null   int64  
 6   EASE-MENT                       0 non-null      float64
 7   BUILDING_CLASS_AT_PRESENT       3151 non-null   object 
 8   ADDRESS                         3151 non-null   object 
 9   APARTMENT_NUMBER                1 non-null      object 
 10  ZIP_CODE                        3151 non-null   float64
 11  RESIDENTIAL_UNITS               3151 non-null   float64
 12  COMMERCIAL_UNITS                

In [26]:
X['SALE_DATE'].dtype
# What is the type of column 'SALE_DATE'?

dtype('O')

In [27]:
# SALE_DATE is an object. Convert to datetime. 

X['SALE_DATE'] = pd.to_datetime(X['SALE_DATE'], infer_datetime_format=False)

X.head(2)

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_DATE
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,nan,A9,4832 BAY PARKWAY,NaN,"11,230",1,0,1,"6,800","1,325","1,930",1,A9,2019-01-01
61,4,OTHER,01 ONE FAMILY DWELLINGS,1,7918,72,nan,A1,80-23 232ND STREET,NaN,"11,427",1,0,1,"4,000","2,001","1,940",1,A1,2019-01-01


In [28]:
X.info()
# Verify again.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3151 entries, 44 to 23035
Data columns (total 20 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   BOROUGH                         3151 non-null   object        
 1   NEIGHBORHOOD                    3151 non-null   object        
 2   BUILDING_CLASS_CATEGORY         3151 non-null   object        
 3   TAX_CLASS_AT_PRESENT            3151 non-null   object        
 4   BLOCK                           3151 non-null   int64         
 5   LOT                             3151 non-null   int64         
 6   EASE-MENT                       0 non-null      float64       
 7   BUILDING_CLASS_AT_PRESENT       3151 non-null   object        
 8   ADDRESS                         3151 non-null   object        
 9   APARTMENT_NUMBER                1 non-null      object        
 10  ZIP_CODE                        3151 non-null   float64       
 11  RE

In [30]:
# Now that we have created our features and target 
# Create Train data only within Jan-Mar 2019 parameters
X_train = X[(X['SALE_DATE'] > '2019-01-01') & (X['SALE_DATE'] < '2019-04-01')]
y_train = y[y.index.isin(X_train.index)]

In [33]:
X['SALE_DATE'][0:]
# Verify parameters w/ slicing.
# This takes a look at the 'head' and 'tail' end of column 'SALE_DATE'

44      2019-01-01
61      2019-01-01
78      2019-01-02
108     2019-01-02
111     2019-01-02
           ...    
23029   2019-04-30
23031   2019-04-30
23032   2019-04-30
23033   2019-04-30
23035   2019-04-30
Name: SALE_DATE, Length: 3151, dtype: datetime64[ns]

In [35]:
# Create Test data only with April 2019
X_test = X[(X['SALE_DATE'] > '2019-04-01') & (X['SALE_DATE'] < '2019-05-01')]
y_test = y[y.index.isin(X_test.index)]

- [ ] Do one-hot encoding of categorical features

#OneHotEncoding??

>>"One-hot encoding" adds a dimension for each unique value of each categorical feature. So, it may not be a good choice for "high cardinality" categoricals that have dozens, hundreds, or thousands of unique values.

Cardinality means the number of unique values that a feature has:

In mathematics, the cardinality of a set means the number of its elements. For example, the set A = {2, 4, 6} contains 3 elements, and therefore A has a cardinality of 3.

In [36]:
# Categorical features can be shown with .describe(exclude='number')
# Non-numeric values
X.describe(exclude='number').columns
X_cat = X.describe(exclude='number')
# Assign X_cat for all usable categorical features in X (DataFrame)

Index(['BOROUGH', 'NEIGHBORHOOD', 'BUILDING_CLASS_CATEGORY',
       'TAX_CLASS_AT_PRESENT', 'BUILDING_CLASS_AT_PRESENT', 'ADDRESS',
       'APARTMENT_NUMBER', 'LAND_SQUARE_FEET',
       'BUILDING_CLASS_AT_TIME_OF_SALE', 'SALE_DATE'],
      dtype='object')

In [42]:
X_cat = X.describe(exclude='number')

In [44]:
# Let's look at relationships between categorical features.
X_cat.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6 entries, count to last
Data columns (total 10 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   BOROUGH                         4 non-null      object
 1   NEIGHBORHOOD                    4 non-null      object
 2   BUILDING_CLASS_CATEGORY         4 non-null      object
 3   TAX_CLASS_AT_PRESENT            4 non-null      object
 4   BUILDING_CLASS_AT_PRESENT       4 non-null      object
 5   ADDRESS                         4 non-null      object
 6   APARTMENT_NUMBER                4 non-null      object
 7   LAND_SQUARE_FEET                4 non-null      object
 8   BUILDING_CLASS_AT_TIME_OF_SALE  4 non-null      object
 9   SALE_DATE                       6 non-null      object
dtypes: object(10)
memory usage: 688.0+ bytes


In [ ]:
"""All object data types"""

In [53]:
# BOROUGH                  

# Import Class
from sklearn.preprocessing import OneHotEncoder
#Instantiate
ohe = OneHotEncoder()
#Fit Transformer to data
ohe.fit(X_train[['BOROUGH']])
#Transform data
borough_train_trans = ohe.transform(X_train[['BOROUGH']]).toarray()

print(type(borough_train_trans), borough_train_trans.shape,'\n' , borough_train_trans)

<class 'numpy.ndarray'> (2505, 5) 
 [[0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 ...
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0.]]


In [72]:
# NEIGHBORHOOD

# Import Class
from sklearn.preprocessing import OneHotEncoder
#Instantiate Class
ohe = OneHotEncoder(sparse=True)
# Look an the defaults of OneHotEncoder
# Fit transformer to data
ohe.fit(X_train[['NEIGHBORHOOD']])
# Transform data
neighborhood_train_trans = ohe.transform(X_train[['NEIGHBORHOOD']]).toarray()

print( 'Neighborhood type :', type(neighborhood_train_trans), '\n',
      'Neighborhood Shape :', neighborhood_train_trans.shape, '\n', 
      'Neighborhood after OneHotEncoder Transformation :', '\n',
      neighborhood_train_trans)

Neighborhood type : <class 'numpy.ndarray'> 
 Neighborhood Shape : (2505, 7) 
 Neighborhood after OneHotEncoder Transformation : 
 [[0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]


In [123]:
X_train['NEIGHBORHOOD'].value_counts()

OTHER                 2358
FLUSHING-NORTH          77
EAST NEW YORK           22
FOREST HILLS            17
BOROUGH PARK            12
ASTORIA                 11
BEDFORD STUYVESANT       8
Name: NEIGHBORHOOD, dtype: int64

In [73]:
# BUILDING_CLASS_CATEGORY         

# Import Class
from sklearn.preprocessing import OneHotEncoder
# Instantiate Class
ohe = OneHotEncoder()
# Fit transformer to data
ohe.fit(X_train[['BUILDING_CLASS_CATEGORY']])
# Transform data
buildingclasscat_train_trans = ohe.transform(X_train[['BUILDING_CLASS_CATEGORY']]).toarray()

print( 'Building Class Category type :', type(buildingclasscat_train_trans), '\n',
      'Building Class Shape :', buildingclasscat_train_trans.shape, '\n', 
      'Building Class Category after OneHotEncoder Transformation :', '\n',
      buildingclasscat_train_trans)

Building Class Category type : <class 'numpy.ndarray'> 
 Building Class Shape : (2505, 1) 
 Building Class Category after OneHotEncoder Transformation : 
 [[1.]
 [1.]
 [1.]
 ...
 [1.]
 [1.]
 [1.]]


In [79]:
print('This is why the above cell has only "1" :', '\n',X_train['BUILDING_CLASS_CATEGORY'])

This is why the above cell has only "1" : 
 78       01 ONE FAMILY DWELLINGS
108      01 ONE FAMILY DWELLINGS
111      01 ONE FAMILY DWELLINGS
120      01 ONE FAMILY DWELLINGS
121      01 ONE FAMILY DWELLINGS
                  ...           
18129    01 ONE FAMILY DWELLINGS
18130    01 ONE FAMILY DWELLINGS
18132    01 ONE FAMILY DWELLINGS
18134    01 ONE FAMILY DWELLINGS
18147    01 ONE FAMILY DWELLINGS
Name: BUILDING_CLASS_CATEGORY, Length: 2505, dtype: object


In [80]:
# TAX_CLASS_AT_PRESENT

# Import Class
from sklearn.preprocessing import OneHotEncoder
# Instatiate Class
ohe = OneHotEncoder()
# Fit transformer to data
ohe.fit(X_train[['TAX_CLASS_AT_PRESENT']])
# Transform data
taxclasspresent_train_trans = ohe.transform(X_train[['TAX_CLASS_AT_PRESENT']]).toarray()

print('Tax Class at Present type :', type(taxclasspresent_train_trans), '\n',
      'Tax Class at Present Shape :', taxclasspresent_train_trans.shape, '\n', 
      'Tax Class at Present after OneHotEncoder Transformation :', '\n',
      taxclasspresent_train_trans)

Tax Class at Present type : <class 'numpy.ndarray'> 
 Tax Class at Present Shape : (2505, 2) 
 Tax Class at Present after OneHotEncoder Transformation : 
 [[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]


In [81]:
print('Value Counts for Tax Class at present :', '\n', X_train['TAX_CLASS_AT_PRESENT'].value_counts())

Value Counts for Tax Class at present : 
 1     2474
1D      31
Name: TAX_CLASS_AT_PRESENT, dtype: int64


In [82]:
# BUILDING_CLASS_AT_PRESENT

# Import Class
from sklearn.preprocessing import OneHotEncoder
# Instantiate Class
ohe = OneHotEncoder(sparse=True)
# Fit transformer to data
ohe.fit(X_train[['BUILDING_CLASS_AT_PRESENT']])
# Transform data
buildingclasspresent_train_trans = ohe.transform(X_train[['BUILDING_CLASS_AT_PRESENT']]).toarray()

print('Building Class at Present :', type(buildingclasspresent_train_trans), '\n',
      'Building Class at Present :', buildingclasspresent_train_trans.shape, '\n', 
      'Building Class at Present after OneHotEncoder Transformation :', '\n',
      buildingclasspresent_train_trans)

Building Class at Present : <class 'numpy.ndarray'> 
 Building Class at Present : (2505, 13) 
 Building Class at Present after OneHotEncoder Transformation : 
 [[0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [84]:
# ADDRESS   

# Import Class
from sklearn.preprocessing import OneHotEncoder
# Instantiate Class
ohe = OneHotEncoder()
# Fit transformer to data
ohe.fit(X_train[['ADDRESS']])
# Transform data
address_train_trans = ohe.transform(X_train[['ADDRESS']]).toarray()

print('Address type:', type(buildingclasspresent_train_trans), '\n',
      'Address shape:', buildingclasspresent_train_trans.shape, '\n', 
      'Address after OneHotEncoder Transformation:', '\n',
      buildingclasspresent_train_trans)

Address type: <class 'numpy.ndarray'> 
 Address shape: (2505, 13) 
 Address after OneHotEncoder Transformation: 
 [[0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [85]:
# APARTMENT_NUMBER           

# Import Class
from sklearn.preprocessing import OneHotEncoder
# Instantiate Class
ohe = OneHotEncoder()
# Fit transformer to data
ohe.fit(X_train[['APARTMENT_NUMBER']])
# Transform data
aptnum_train_trans = ohe.transform(X_train[['APARTMENT_NUMBER']]).toarray()

print('Apartment number type:', type(aptnum_train_trans), '\n',
      'Apartment number shape:', aptnum_train_trans.shape, '\n', 
      'Apartment number after OneHotEncoder Transformation:', '\n',
      aptnum_train_trans)

ValueError: ignored

In [87]:
X_train.drop('APARTMENT_NUMBER', inplace=True, axis=1)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [92]:
print('Apartment Number column dropped since it contains Nan values.','\n','\n',X_train.columns)

Apartment Number column dropped since it contains Nan values. 
 
 Index(['BOROUGH', 'NEIGHBORHOOD', 'BUILDING_CLASS_CATEGORY',
       'TAX_CLASS_AT_PRESENT', 'BLOCK', 'LOT', 'EASE-MENT',
       'BUILDING_CLASS_AT_PRESENT', 'ADDRESS', 'ZIP_CODE', 'RESIDENTIAL_UNITS',
       'COMMERCIAL_UNITS', 'TOTAL_UNITS', 'LAND_SQUARE_FEET',
       'GROSS_SQUARE_FEET', 'YEAR_BUILT', 'TAX_CLASS_AT_TIME_OF_SALE',
       'BUILDING_CLASS_AT_TIME_OF_SALE', 'SALE_DATE'],
      dtype='object')


In [93]:
# LAND_SQUARE_FEET                

# Import Class
from sklearn.preprocessing import OneHotEncoder
# Instantiate Class
ohe = OneHotEncoder()
# Fit transformer to data
ohe.fit(X_train[['LAND_SQUARE_FEET']])
# Transform data
landsqfeet_train_trans = ohe.transform(X_train[['LAND_SQUARE_FEET']]).toarray()

print('Land Square Feet type:', type(landsqfeet_train_trans), '\n',
      'Land Square Feet shape:', landsqfeet_train_trans.shape, '\n', 
      'Land Square Feet after OneHotEncoder Transformation:', '\n',
      landsqfeet_train_trans)

Land Square Feet type: <class 'numpy.ndarray'> 
 Land Square Feet shape: (2505, 886) 
 Land Square Feet after OneHotEncoder Transformation: 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [96]:
print('High Cardinality (Length:886) :','\n',X_train['LAND_SQUARE_FEET'].value_counts())
#886 different values for Land Square Feet. 


High Cardinality (Length:886) : 
 4,000    233
2,000    187
2,500    145
1,800     95
3,000     92
        ... 
3,361      1
4,454      1
2,402      1
2,275      1
4,268      1
Name: LAND_SQUARE_FEET, Length: 886, dtype: int64


In [97]:
# BUILDING_CLASS_AT_TIME_OF_SALE  

# Import Class
from sklearn.preprocessing import OneHotEncoder
# Instantiate Class
ohe = OneHotEncoder()
# Fit transformer to data
ohe.fit(X_train[['BUILDING_CLASS_AT_TIME_OF_SALE']])
# Transform data
buildclassattimeofsale_train_trans = ohe.transform(X_train[['BUILDING_CLASS_AT_TIME_OF_SALE']]).toarray()

print('Building Class at time of Sale type:', type(buildclassattimeofsale_train_trans), '\n',
      'Building Class at time of Sale shape:', buildclassattimeofsale_train_trans.shape, '\n', 
      'Building Class at time of Sale after OneHotEncoder Transformation:', '\n',
      buildclassattimeofsale_train_trans)

Building Class at time of Sale type: <class 'numpy.ndarray'> 
 Building Class at time of Sale shape: (2505, 11) 
 Building Class at time of Sale after OneHotEncoder Transformation: 
 [[0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [98]:
X_train['BUILDING_CLASS_AT_TIME_OF_SALE'].value_counts()

A1    918
A5    779
A2    413
A9    192
A0     67
S1     39
A3     38
A8     31
A6     14
A4     13
S0      1
Name: BUILDING_CLASS_AT_TIME_OF_SALE, dtype: int64

In [99]:
# SALE_DATE                       

# Import Class
from sklearn.preprocessing import OneHotEncoder
# Instantiate Class
ohe = OneHotEncoder()
# Fit transformer to data
ohe.fit(X_train[['SALE_DATE']])
# Transform data
saledate_train_trans = ohe.transform(X_train[['SALE_DATE']]).toarray()

print('Sale Date type:', type(saledate_train_trans), '\n',
      'Sale Date shape:', saledate_train_trans.shape, '\n', 
      'Sale Date after OneHotEncoder Transformation:', '\n',
      saledate_train_trans)

Sale Date type: <class 'numpy.ndarray'> 
 Sale Date shape: (2505, 67) 
 Sale Date after OneHotEncoder Transformation: 
 [[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [102]:
print('High Cardinality (Length:67) :','\n','\n',X_train['SALE_DATE'].value_counts())

High Cardinality (Length:67) : 
 
 2019-01-31    78
2019-03-29    62
2019-02-28    58
2019-01-15    57
2019-01-24    56
              ..
2019-02-18     3
2019-03-30     1
2019-03-17     1
2019-03-09     1
2019-02-17     1
Name: SALE_DATE, Length: 67, dtype: int64


- [ ] Feature selection with **`SelectKbest` (Univariate Feature Selection)**

*Univariate is used in statistics to describe a type of data which consisits of obserbations on oly a single characteristic or attribute*

> Tells which features are most closely associated with your target.

>Use Ski-Kit Learn

In [117]:
features = X_train.columns
n = len(features)
print(f'Number of features: {n:,.0f}')

Number of features: 18


In [118]:
# How many ways to choose 1 to n features?
from math import factorial

def n_choose_k(n, k):
    return factorial(n)/(factorial(k)*factorial(n-k))

combinations = sum(n_choose_k(n,k) for k in range(1,n+1))
print(f'Every possible combination of "features" : {combinations:,.0f}')

Every possible combination of "features" : 262,143


In [ ]:
# y_train
# y_test
X_train.drop('EASE-MENT', axis=1, inplace=True)
X_test.drop('EASE-MENT', axis=1, inplace=True)
X_train_select = X_train.select_dtypes(include='number')
X_test_select = X_test.select_dtypes(include='number')

In [122]:
X_train['BUILDING_CLASS_CATEGORY'].value_counts()

01 ONE FAMILY DWELLINGS    2505
Name: BUILDING_CLASS_CATEGORY, dtype: int64

In [130]:
# Select the 6 features that best correlate with the target
# SelectKBest has a similar API to what we've seen before.

from sklearn.feature_selection import f_regression, SelectKBest

selector = SelectKBest(score_func=f_regression, k=6)
# First parameter to set is how many features you need that correlate with target.

#X_train.drop('NEIGHBORHOOD', axis=1, inplace=True)
#X_test.drop('NEIGHBORHOOD', axis=1, inplace=True)

# IMPORTANT!
# .fit_transform() on the train set
# .transform() on test set
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)
X_train_selected.shape, X_test_selected.shape


ValueError: ignored

In [ ]:
### Which features were selected?
all_names = X_train.columns
selected_mask = selector.get_support()
selected_names = all_names[selected_mask]
unselected_names = all_names[~selected_mask]

print('Features selected:')
for name in selected_names:
    print(name)

print('\n')
print('Features not selected:')
for name in unselected_names:
    print(name)

In [ ]:
### How many features should be selected?
### You can try a range of values for k,
### then choose the model with the best score.
### If multiple models "tie" for the best score,
### choose the simplest model.
### You decide what counts as a tie!

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

for k in range(1, len(X_train.columns)+1):

    print(f'{k} features')

    selector = SelectKBest(score_func=f_regression, k=k)
    X_train_selected = selector.fit_transform(X_train, y_train)
    X_test_selected = selector.transform(X_test)

    model = LinearRegression()
    model.fit(X_train_selected, y_train)
    y_pred = model.predict(X_test_selected)

    mae = mean_absolute_error(y_test, y_pred)
    print(f'Test MAE: ${mae:,.0f} \n')

## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).